In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import math
import time
import os
from collections import deque

class AdvancedFaceDetectionSystem:
    def __init__(self):
        # Initialize MediaPipe components
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_face_mesh = mp.solutions.face_mesh
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
        # MediaPipe Face Detection (more accurate than Haar cascades)
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=0,  # 0 for close-range (2m), 1 for full-range (5m)
            min_detection_confidence=0.5
        )
        
        # MediaPipe Face Mesh (468 landmarks)
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=5,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # Age/Gender Detection Models
        self.age_net = None
        self.gender_net = None
        self.load_age_gender_models()
        
        # Feature toggles
        self.show_landmarks = False
        self.show_age_gender = False
        self.show_confidence = True
        self.show_face_mesh = False
        
        # AR Filter system
        self.current_filter = "none"  # none, sunglasses, mustache, hat
        self.create_ar_filters()
        
        # Performance tracking
        self.fps_counter = deque(maxlen=30)
        self.frame_count = 0
        self.start_time = time.time()
        
        # Age and gender lists
        self.age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        self.gender_list = ['Male', 'Female']
    
    def load_age_gender_models(self):
        """Load pre-downloaded age and gender detection models"""
        try:
            # Load age detection model
            if os.path.exists('models/age_net.caffemodel') and os.path.exists('models/age_deploy.prototxt'):
                self.age_net = cv2.dnn.readNet('models/age_net.caffemodel', 'models/age_deploy.prototxt')
                print("✅ Age detection model loaded successfully")
            else:
                print("❌ Age model files not found in models/ directory")
            
            # Load gender detection model  
            if os.path.exists('models/gender_net.caffemodel') and os.path.exists('models/gender_deploy.prototxt'):
                self.gender_net = cv2.dnn.readNet('models/gender_net.caffemodel', 'models/gender_deploy.prototxt')
                print("✅ Gender detection model loaded successfully")
            else:
                print("❌ Gender model files not found in models/ directory")
                
        except Exception as e:
            print(f"❌ Error loading age/gender models: {e}")
            print("Make sure model files are in the 'models/' directory")
    
    def create_ar_filters(self):
        """Create AR filter assets"""
        # Create sunglasses filter
        self.sunglasses = np.zeros((50, 150, 4), dtype=np.uint8)
        cv2.rectangle(self.sunglasses, (0, 0), (149, 49), (0, 0, 0, 200), -1)  # Dark lenses
        cv2.rectangle(self.sunglasses, (0, 0), (149, 49), (255, 255, 255, 255), 3)  # White frame
        cv2.line(self.sunglasses, (75, 20), (75, 30), (255, 255, 255, 255), 4)  # Bridge
        
        # Create mustache filter
        self.mustache = np.zeros((40, 100, 4), dtype=np.uint8)
        cv2.ellipse(self.mustache, (50, 25), (45, 15), 0, 0, 180, (101, 67, 33, 255), -1)
        cv2.ellipse(self.mustache, (50, 25), (45, 15), 0, 0, 180, (139, 90, 43, 255), 3)
        
        # Create hat filter
        self.hat = np.zeros((80, 120, 4), dtype=np.uint8)
        cv2.rectangle(self.hat, (10, 60), (110, 75), (255, 0, 0, 255), -1)  # Hat brim
        cv2.rectangle(self.hat, (25, 10), (95, 65), (139, 0, 0, 255), -1)   # Hat top
        
        print("✅ AR filters created successfully")
    
    def detect_faces_mediapipe(self, frame):
        """Advanced face detection using MediaPipe"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(rgb_frame)
        
        faces = []
        if results.detections:
            for detection in results.detections:
                # Get bounding box
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                
                x = max(0, int(bbox.xmin * w))
                y = max(0, int(bbox.ymin * h))
                width = min(w - x, int(bbox.width * w))
                height = min(h - y, int(bbox.height * h))
                
                # Get confidence score
                confidence = detection.score[0]
                
                faces.append({
                    'bbox': (x, y, width, height),
                    'confidence': confidence,
                    'keypoints': self.extract_face_keypoints(detection, w, h)
                })
        
        return faces
    
    def extract_face_keypoints(self, detection, frame_width, frame_height):
        """Extract facial keypoints from MediaPipe detection"""
        keypoints = {}
        if detection.location_data.relative_keypoints:
            keypoint_names = ['right_eye', 'left_eye', 'nose_tip', 'mouth_center', 'right_ear', 'left_ear']
            for i, keypoint in enumerate(detection.location_data.relative_keypoints):
                if i < len(keypoint_names):
                    keypoints[keypoint_names[i]] = (
                        int(keypoint.x * frame_width),
                        int(keypoint.y * frame_height)
                    )
        return keypoints
    
    def get_facial_landmarks(self, frame):
        """Get 468 facial landmarks using MediaPipe Face Mesh"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        
        landmarks_data = []
        if results.multi_face_landmarks:
            h, w, _ = frame.shape
            for face_landmarks in results.multi_face_landmarks:
                landmarks = []
                for landmark in face_landmarks.landmark:
                    x = int(landmark.x * w)
                    y = int(landmark.y * h)
                    landmarks.append((x, y))
                landmarks_data.append(landmarks)
        
        return landmarks_data, results
    
    def predict_age_gender(self, face_roi):
        """Predict age and gender using CNN models"""
        if self.age_net is None or self.gender_net is None:
            return "Model Error", "Model Error", 0.0
        
        if face_roi.size == 0:
            return "Invalid Face", "Invalid Face", 0.0
        
        try:
            # Preprocess face region
            blob = cv2.dnn.blobFromImage(
                face_roi, 1.0, (227, 227),
                (78.4263377603, 87.7689143744, 114.895847746)
            )
            
            # Age prediction
            self.age_net.setInput(blob)
            age_predictions = self.age_net.forward()
            age_idx = age_predictions[0].argmax()
            age_confidence = age_predictions[0].max()
            predicted_age = self.age_list[age_idx]
            
            # Gender prediction
            self.gender_net.setInput(blob)
            gender_predictions = self.gender_net.forward()
            gender_idx = gender_predictions[0].argmax()
            gender_confidence = gender_predictions[0].max()
            predicted_gender = self.gender_list[gender_idx]
            
            # Combined confidence
            combined_confidence = (age_confidence + gender_confidence) / 2
            
            return predicted_age, predicted_gender, combined_confidence
            
        except Exception as e:
            print(f"Prediction error: {e}")
            return "Error", "Error", 0.0
    
    def apply_sunglasses_filter(self, frame, landmarks):
        """Apply sunglasses AR filter based on facial landmarks"""
        if len(landmarks) < 468:
            return
        
        try:
            # Get eye positions (MediaPipe landmark indices)
            left_eye_center = landmarks[33]   # Left eye center
            right_eye_center = landmarks[362] # Right eye center
            
            # Calculate sunglasses parameters
            eye_distance = math.sqrt(
                (right_eye_center[0] - left_eye_center[0])**2 + 
                (right_eye_center[1] - left_eye_center[1])**2
            )
            
            # Calculate center point between eyes
            center_x = (left_eye_center[0] + right_eye_center[0]) // 2
            center_y = (left_eye_center[1] + right_eye_center[1]) // 2
            
            # Scale sunglasses based on eye distance
            scale_factor = eye_distance / 100
            new_width = int(self.sunglasses.shape[1] * scale_factor)
            new_height = int(self.sunglasses.shape[0] * scale_factor)
            
            if new_width > 10 and new_height > 10:
                # Resize sunglasses
                resized_sunglasses = cv2.resize(self.sunglasses, (new_width, new_height))
                
                # Calculate position
                start_x = center_x - new_width // 2
                start_y = center_y - new_height // 2
                
                # Apply overlay
                self.overlay_image_alpha(frame, resized_sunglasses, start_x, start_y)
                
        except Exception as e:
            print(f"Sunglasses filter error: {e}")
    
    def apply_mustache_filter(self, frame, landmarks):
        """Apply mustache AR filter"""
        if len(landmarks) < 468:
            return
        
        try:
            # Get nose and mouth positions
            nose_bottom = landmarks[2]    # Nose bottom
            mouth_top = landmarks[12]     # Mouth top
            
            # Calculate mustache position
            mustache_center_x = (nose_bottom[0] + mouth_top[0]) // 2
            mustache_center_y = (nose_bottom[1] + mouth_top[1]) // 2
            
            # Scale based on face width
            face_width = abs(landmarks[454][0] - landmarks[234][0])  # Face contour points
            scale_factor = face_width / 200
            
            new_width = int(self.mustache.shape[1] * scale_factor)
            new_height = int(self.mustache.shape[0] * scale_factor)
            
            if new_width > 10 and new_height > 10:
                resized_mustache = cv2.resize(self.mustache, (new_width, new_height))
                
                start_x = mustache_center_x - new_width // 2
                start_y = mustache_center_y - new_height // 2
                
                self.overlay_image_alpha(frame, resized_mustache, start_x, start_y)
                
        except Exception as e:
            print(f"Mustache filter error: {e}")
    
    def apply_hat_filter(self, frame, landmarks):
        """Apply hat AR filter"""
        if len(landmarks) < 468:
            return
        
        try:
            # Get forehead position
            forehead_center = landmarks[10]  # Forehead center
            
            # Calculate hat position
            face_width = abs(landmarks[454][0] - landmarks[234][0])
            scale_factor = face_width / 150
            
            new_width = int(self.hat.shape[1] * scale_factor)
            new_height = int(self.hat.shape[0] * scale_factor)
            
            if new_width > 10 and new_height > 10:
                resized_hat = cv2.resize(self.hat, (new_width, new_height))
                
                start_x = forehead_center[0] - new_width // 2
                start_y = forehead_center[1] - new_height
                
                self.overlay_image_alpha(frame, resized_hat, start_x, start_y)
                
        except Exception as e:
            print(f"Hat filter error: {e}")
    
    def overlay_image_alpha(self, background, overlay, x, y):
        """Overlay image with alpha channel onto background"""
        if overlay.shape[2] != 4:
            return
        
        h, w = overlay.shape[:2]
        
        # Ensure overlay is within frame bounds
        if x < 0 or y < 0 or x + w > background.shape[1] or y + h > background.shape[0]:
            return
        
        # Extract alpha channel
        alpha = overlay[:, :, 3] / 255.0
        
        # Apply overlay for each color channel
        for c in range(3):
            background[y:y+h, x:x+w, c] = (
                alpha * overlay[:, :, c] + 
                (1 - alpha) * background[y:y+h, x:x+w, c]
            )
    
    def draw_landmarks(self, frame, landmarks):
        """Draw 468 facial landmarks"""
        for point in landmarks:
            cv2.circle(frame, point, 1, (0, 255, 0), -1)
    
    def draw_face_mesh(self, frame, face_mesh_results):
        """Draw 3D face mesh"""
        if face_mesh_results.multi_face_landmarks:
            for face_landmarks in face_mesh_results.multi_face_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, 
                    face_landmarks,
                    self.mp_face_mesh.FACEMESH_CONTOURS,
                    None,
                    self.mp_drawing_styles.get_default_face_mesh_contours_style()
                )
    
    def draw_ui_controls(self, frame):
        """Draw UI controls and information"""
        height, width = frame.shape[:2]
        
        # Create semi-transparent overlay for controls
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, height-150), (400, height-10), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        
        # Control instructions
        controls = [
            "CONTROLS:",
            "S - Toggle Sunglasses",
            "M - Toggle Mustache", 
            "H - Toggle Hat",
            "L - Toggle Landmarks",
            "F - Toggle Face Mesh",
            "A - Toggle Age/Gender",
            "C - Toggle Confidence",
            "Q - Quit"
        ]
        
        for i, control in enumerate(controls):
            color = (0, 255, 255) if i == 0 else (255, 255, 255)
            cv2.putText(frame, control, (15, height-140+i*15), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        
        # Status display
        status_y = 30
        status_info = [
            f"Filter: {self.current_filter}",
            f"Landmarks: {'ON' if self.show_landmarks else 'OFF'}",
            f"Age/Gender: {'ON' if self.show_age_gender else 'OFF'}",
            f"FPS: {np.mean(self.fps_counter):.1f}" if self.fps_counter else "FPS: --"
        ]
        
        for i, status in enumerate(status_info):
            cv2.putText(frame, status, (width-200, status_y+i*25), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    def process_frame(self, frame):
        """Process a single frame with all features"""
        frame_start = time.time()
        
        # 1. Detect faces using MediaPipe
        faces = self.detect_faces_mediapipe(frame)
        
        # 2. Get facial landmarks
        landmarks_data, face_mesh_results = self.get_facial_landmarks(frame)
        
        # 3. Process each detected face
        for i, face in enumerate(faces):
            x, y, w, h = face['bbox']
            confidence = face['confidence']
            
            # Draw face bounding box
            color = (0, 255, 0) if confidence > 0.7 else (0, 255, 255)
            thickness = 2 if confidence > 0.7 else 1
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, thickness)
            
            # Show confidence if enabled
            if self.show_confidence:
                cv2.putText(frame, f'Confidence: {confidence:.2f}', 
                          (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            # Age and gender prediction
            if self.show_age_gender:
                face_roi = frame[y:y+h, x:x+w]
                if face_roi.size > 0:
                    age, gender, ag_confidence = self.predict_age_gender(face_roi)
                    text = f'{gender}, {age}'
                    cv2.putText(frame, text, (x, y-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    
                    if self.show_confidence:
                        cv2.putText(frame, f'A/G Conf: {ag_confidence:.2f}', 
                                  (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        
        # 4. Apply AR filters and landmarks
        if landmarks_data:
            for landmarks in landmarks_data:
                # Show landmarks if enabled
                if self.show_landmarks:
                    self.draw_landmarks(frame, landmarks)
                
                # Apply AR filters
                if self.current_filter == "sunglasses":
                    self.apply_sunglasses_filter(frame, landmarks)
                elif self.current_filter == "mustache":
                    self.apply_mustache_filter(frame, landmarks)
                elif self.current_filter == "hat":
                    self.apply_hat_filter(frame, landmarks)
        
        # 5. Show face mesh if enabled
        if self.show_face_mesh:
            self.draw_face_mesh(frame, face_mesh_results)
        
        # 6. Draw UI controls
        self.draw_ui_controls(frame)
        
        # Update FPS
        frame_time = time.time() - frame_start
        if frame_time > 0:
            self.fps_counter.append(1.0 / frame_time)
        
        return frame
    
    def run_detection(self):
        """Main detection loop"""
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        print("🚀 Advanced Face Detection System Started!")
        print("🎯 Features: MediaPipe Detection | 468 Landmarks | Age/Gender | AR Filters")
        print("📋 Press keys to toggle features (see on-screen controls)")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to read from camera")
                break
            
            # Process frame with all features
            frame = self.process_frame(frame)
            
            # Display frame
            cv2.imshow('Advanced Face Detection System', frame)
            
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('s'):
                self.current_filter = "sunglasses" if self.current_filter != "sunglasses" else "none"
                print(f"Filter: {self.current_filter}")
            elif key == ord('m'):
                self.current_filter = "mustache" if self.current_filter != "mustache" else "none"
                print(f"Filter: {self.current_filter}")
            elif key == ord('h'):
                self.current_filter = "hat" if self.current_filter != "hat" else "none"
                print(f"Filter: {self.current_filter}")
            elif key == ord('l'):
                self.show_landmarks = not self.show_landmarks
                print(f"Landmarks: {'ON' if self.show_landmarks else 'OFF'}")
            elif key == ord('f'):
                self.show_face_mesh = not self.show_face_mesh
                print(f"Face Mesh: {'ON' if self.show_face_mesh else 'OFF'}")
            elif key == ord('a'):
                self.show_age_gender = not self.show_age_gender
                print(f"Age/Gender: {'ON' if self.show_age_gender else 'OFF'}")
            elif key == ord('c'):
                self.show_confidence = not self.show_confidence
                print(f"Confidence: {'ON' if self.show_confidence else 'OFF'}")
        
        cap.release()
        cv2.destroyAllWindows()
        
        # Print session summary
        total_time = time.time() - self.start_time
        avg_fps = np.mean(self.fps_counter) if self.fps_counter else 0
        print(f"\n📊 Session Summary:")
        print(f"   Duration: {total_time:.1f}s")
        print(f"   Average FPS: {avg_fps:.1f}")
        print(f"   Total Frames: {self.frame_count}")

# USAGE
if __name__ == "__main__":
    # Create and run the advanced face detection system
    system = AdvancedFaceDetectionSystem()
    system.run_detection()

"""
🎯 FEATURES IMPLEMENTED:

✅ MediaPipe Face Detection (10x more accurate than Haar cascades)
✅ Real-time confidence scores for each detection
✅ 468 facial landmarks with precise tracking
✅ Age & Gender detection using CNN models
✅ 3D face mesh visualization
✅ AR Filters: Sunglasses, Mustache, Hat
✅ Interactive controls with real-time switching
✅ Performance monitoring (FPS tracking)
✅ Alpha blending for realistic filter effects
✅ Adaptive filter sizing based on face dimensions

🎮 CONTROLS:
S - Toggle Sunglasses Filter
M - Toggle Mustache Filter  
H - Toggle Hat Filter
L - Show/Hide 468 Landmarks
F - Toggle 3D Face Mesh
A - Enable/Disable Age/Gender Detection
C - Show/Hide Confidence Scores
Q - Quit Application

🔧 REQUIREMENTS:
- Place age/gender models in 'models/' directory:
  * age_net.caffemodel
  * age_deploy.prototxt  
  * gender_net.caffemodel
  * gender_deploy.prototxt

📦 INSTALL:
pip install opencv-python mediapipe numpy
"""